## Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import datetime
from PriceFetcher import PriceFetcher
from MarketCorrections import MarketCorrections
from Utilities import period_max_drawdown

## Importing Market Data

In [ ]:
# pf = PriceFetcher(assets=["SPY", "HYG"])
# pf.fetch()
# df_px = pf.prices

In [ ]:
df_spy = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "spy")
df_agg = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "agg")
df_hyg = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "hyg")
df_tlt = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "tlt")
df_gld = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "gld")
df_buffer_010 = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "mqu1bslq")
df_buffer_020 = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "mquslblr")
df_buffer_100 = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "mqu1pplr")
df_sv_hedged_income = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "sv_hedged_income")
df_sv_hedged_balanced = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "sv_hedged_balanced")
df_sv_hedged_enhanced_growth = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "sv_hedged_enhanced_growth")
df_sv_equity_buffer = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "sv_equity_buffer")
df_sv_equity_buffer_growth = pd.read_excel("data/bufr_bufd_mquslblr.xlsx", "sv_equity_buffer_growth")

df_px = (
    df_buffer_100
        .merge(df_buffer_010, how="left", on="date")
        .merge(df_buffer_020, how="left", on="date")
        .merge(df_spy, how="left", on="date")
        .merge(df_agg, how="left", on="date")
        .merge(df_hyg, how="left", on="date")
        .merge(df_tlt, how="left", on="date")
        .merge(df_gld, how="left", on="date")
        .merge(df_sv_hedged_income, how="left", on="date")
        .merge(df_sv_hedged_balanced, how="left", on="date")
        .merge(df_sv_hedged_enhanced_growth, how="left", on="date")
        .merge(df_sv_equity_buffer, how="left", on="date")
        .merge(df_sv_equity_buffer_growth, how="left", on="date")
)
df_px.rename(columns={"mqu1pplr":"buffer_100", "mquslblr":"buffer_020", "mqu1bslq":"buffer_010"}, inplace=True)
df_px = df_px.query("'2007-04-11' <= date & date <= '2024-12-31'").reset_index(drop=True)

In [ ]:
#df_px.info()

## Defining Market Corrections

In [ ]:
mc = MarketCorrections(asset="SPY", correction=-0.05)
df_corrections = mc.corrections

[*********************100%***********************]  1 of 1 completed


In [ ]:
df_corrections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start         33 non-null     object 
 1   end           33 non-null     object 
 2   bottom        33 non-null     object 
 3   drawdown_spy  33 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.2+ KB


## Defining Backtest Parameters

In [ ]:
class FixedWeightBacktester:
    """
    A backtester for a weighted portfolio of assets with daily rebalancing of the weights.

    Attributes
    ----------
    portfolio: dict[str, float]
        Defines the assets and weights in the portfolio.
        
    prices: pd.DataFrame
        Contains the prices of assets as far back as there is data for in Yahoo Finance.
        This is typically the result of the PriceFetcher.fetch() method.
        
    market_corrections: pd.DataFrame
        Contains the start, bottom, end date of prices corrections of a particular asset.
        Typically some kind of broad market index like SPY will be used.  This is usually
        the result of the MarketCorrections class.  It is modified by the calc_period_drawdown
        method to contain the drawdowns of the weighted portfolio during the drawdown periods.
        
    date_start: datetime.date
        The start date of the backtest.
        
    date_end: datetime.date
        The end date of the backtest.

    assets: list[str]
        The component assets in the portfolio of the backtest.  This is extracted from the portfolio.
    
    weights: list[str]
        The weights of the component assets in the portfolio.  This is extracted from the portfolio

    returns: pd.DataFrame
        The prices, daily returns, equity curve, drawdowns of the assets
        and the weighted portfolio that is being backtested.

    cumulative_returns: dict[str, float]
        The cumulative returns for each of the component assets and the weighted portfolio.

    annual_returns: dict[str, float]
        The annualized returns for each of the component assets and the weighted portfolio.

    volatility: dict[str, float]
        The annualized volatility for each of the component assets and the weighted portfolio.

    sharpe_ratio: dict[str, float]
        The annualized sharpe-ratio for each of the component assets and the weighted portfolio.

    sharpe_ratio: dict[str, float]
        The annualized sharpe-ratio for each of the component assets and the weighted portfolio.

    drawdown_max: dict[str, float]
        The maximum for each of the component assets and the weighted portfolio.

    annual_performance: pd.DataFrame
        The performance of the weighted portfolio for each calendar year in the backtest.
    """
    def __init__(self, 
                 portfolio: dict[str, float], 
                 prices: pd.DataFrame,
                 market_corrections: pd.DataFrame,
                 date_start: datetime.date,
                 date_end: datetime.date,
                 frequency_rebalance: str):
        self.portfolio = portfolio
        self.prices = prices        
        self.market_corrections = (
            market_corrections
                .query("@date_start <= start & end <= @date_end").copy()
        )
        """
        portfolio: dict[str, float]
            Defines the assets and weights in the portfolio.
            
        prices: pd.DataFrame
            Contains the prices of assets as far back as there is data for in Yahoo Finance.
            This is typically the result of the PriceFetcher.fetch() method.
            
        market_corrections: pd.DataFrame
            Contains the start, bottom, end date of prices corrections of a particular asset.
            Typically some kind of broad market index like SPY will be used.  This is usually
            the result of the MarketCorrections class.  It is modified by the calc_period_drawdown
            method to contain the drawdowns of the weighted portfolio during the drawdown periods.
            
        date_start: datetime.date
            The start date of the backtest.
            
        date_end: datetime.date
            The end date of the backtest.
        """
        
        self.date_start = date_start
        self.date_end = date_end
        self.frequency_rebalance = frequency_rebalance

        # isolating weights and assets from portfolio
        self.assets = []
        self.weights = []
        for asset, weight in self.portfolio.items():
            self.assets.append(asset)
            self.weights.append(weight)



    def calc_daily_returns(self) -> None:
        """
        Calculates the prices, daily returns, equity curve, drawdowns of the assets
        and the weighted portfolio that is being backtested.
        """
        
        self.returns = self.prices.query("@date_start <= date & date <= @date_end").copy().reset_index(drop=True)

        # calculating component asset daily returns
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            self.returns[ret_col_name] = self.returns[ix_asset].pct_change()
        self.returns.fillna(0, inplace=True)

        # calculating portfolio daily returns
        if self.frequency_rebalance == None:
            cols = []
            for ix_asset in self.assets:
                cols.append("ret_" + ix_asset)
            self.returns["ret_portfolio"] =  np.sum(np.array(self.returns[cols]) * self.weights, axis=1)
        else:
            self.calc_rebalanced_portfolio()

        # calculating equity curve for components and portfolio
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            equity_col_name = "equity_" + ix_asset
            self.returns[equity_col_name] = (1 + self.returns[ret_col_name]).cumprod()
        self.returns["equity_portfolio"] = (1 + self.returns["ret_portfolio"]).cumprod()

        # calculating drawdowns for components and portfolio
        for ix_asset in self.assets:
            equity_col_name = "equity_" + ix_asset
            drawdown_col_name = "drawdown_" + ix_asset
            self.returns[drawdown_col_name] = (self.returns[equity_col_name] / self.returns[equity_col_name].cummax()) - 1
        self.returns["drawdown_portfolio"] = (self.returns["equity_portfolio"] / self.returns["equity_portfolio"].cummax()) - 1


    def calc_rebalanced_portfolio(self) -> None:
        """
        Calculates the daily returns of a rebalanced portfolio.
        """
        df = self.returns[["date"]].copy()
    
        # determining rebalance dates
        if self.frequency_rebalance == "annual":
            self.returns["year"] = self.returns["date"].dt.year
            df["year"] = df["date"].dt.year
            df_date_rebalance = df.groupby(["year"])[["date"]].max().reset_index()
            df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
            self.returns = self.returns.merge(df_date_rebalance, how="left", on=["year"])
        elif self.frequency_rebalance == "semiannual":
            self.returns["year"] = self.returns["date"].dt.year
            self.returns["month"] = self.returns["date"].dt.month
            self.returns["half"] = np.where(self.returns["month"] <= 6, 1, 2)
            df["year"] = df["date"].dt.year
            df["month"] = df["date"].dt.month
            df["half"] = np.where(df["month"] <= 6, 1, 2)
            df_date_rebalance = df.groupby(["year","half"])[["date"]].max().reset_index()
            df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
            self.returns = self.returns.merge(df_date_rebalance, how="left", on=["year", "half"])
        elif self.frequency_rebalance == "quarterly":
            self.returns["year"] = self.returns["date"].dt.year
            self.returns["quarter"] = self.returns["date"].dt.quarter
            df["year"] = df["date"].dt.year
            df["quarter"] = df["date"].dt.quarter
            df_date_rebalance = df.groupby(["year","quarter"])[["date"]].max().reset_index()
            df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
            self.returns = self.returns.merge(df_date_rebalance, how="left", on=["year", "quarter"])
        elif self.frequency_rebalance == "monthly":
            self.returns["year"] = self.returns["date"].dt.year
            self.returns["month"] = self.returns["date"].dt.month
            df["year"] = df["date"].dt.year
            df["month"] = df["date"].dt.month
            df_date_rebalance = df.groupby(["year","month"])[["date"]].max().reset_index()
            df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
            self.returns = self.returns.merge(df_date_rebalance, how="left", on=["year", "month"])
        elif self.frequency_rebalance == "daily":
            self.returns["date_rebalance"] = self.returns["date"]
    
        # initializing values for iteration through self.returns
        before_rebal = {}
        lst_total_value = []
        total_value = 0
        for ix_asset in self.assets:
            before_rebal[ix_asset] = [self.portfolio[ix_asset]]
            total_value += before_rebal[ix_asset][-1]
        lst_total_value.append(total_value)
        after_rebal = {}
        for ix_asset in self.assets:
            after_rebal[ix_asset] = [self.portfolio[ix_asset]]
    
        # iterating through self.returns to calculate portfolio values
        for _ , row in self.returns[1:].iterrows():
            # calculating end-of-day value of each asset allocation
            for ix_asset, _ in before_rebal.items():
                before_rebal[ix_asset].append(after_rebal[ix_asset][-1] * (1 + row["ret_"+ ix_asset]))
            
            # calculating total portfolio value
            total_value = 0
            for ix_asset, _ in before_rebal.items():
                total_value += before_rebal[ix_asset][-1]
            lst_total_value.append(total_value)    
        
            # rebalancing if needed
            if row["date"] == row["date_rebalance"]:
                for ix_asset, _ in after_rebal.items():
                    after_rebal[ix_asset].append(total_value * self.portfolio[ix_asset])
            else:
                for ix_asset, _ in after_rebal.items():
                    after_rebal[ix_asset].append(before_rebal[ix_asset][-1])
    
        # adding columns to self.returns
        df_before_rebal = pd.DataFrame(before_rebal)
        for ix_asset, _ in before_rebal.items():
            df_before_rebal.rename(columns={ix_asset: "before_rebal_" + ix_asset}, inplace=True)
        df_before_rebal
        df_after_rebal = pd.DataFrame(after_rebal)
        for ix_asset, _ in after_rebal.items():
            df_after_rebal.rename(columns={ix_asset: "after_rebal_" + ix_asset}, inplace=True)
        df_after_rebal
        df_total_value = pd.DataFrame({"portfolio_total_value":lst_total_value})
        self.returns = pd.concat([self.returns, df_before_rebal, df_total_value, df_after_rebal], axis=1)
        self.returns["ret_portfolio"] = self.returns["portfolio_total_value"].pct_change()
        self.returns.fillna(0, inplace=True)

    
    def calc_portfolio_statistics(self) -> None:
        """
        Calculates the portfolio statistics and annual performance of the component assets
        and the weighted portfolio being backtested.
        """
        # cumulative return
        self.cumulative_return = {}
        for ix_asset in self.assets:
            equity_col_name = "equity_" + ix_asset
            self.cumulative_return[ix_asset] = (self.returns[equity_col_name].iloc[-1] - 1)
        self.cumulative_return["portfolio"] = self.returns["equity_portfolio"].iloc[-1] - 1
        
        # annual return
        self.annual_return = {}
        for ix_asset in self.assets:
            equity_col_name = "equity_" + ix_asset
            self.annual_return[ix_asset] = (self.returns[equity_col_name].iloc[-1] ** (252/(len(self.returns) - 1)) - 1)
        self.annual_return["portfolio"] = self.returns["equity_portfolio"].iloc[-1] ** (252/(len(self.returns) - 1)) - 1

        # volatility
        self.volatility = {}
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            self.volatility[ix_asset] = self.returns[ret_col_name][1:].std() * np.sqrt(252)
        self.volatility["portfolio"] = self.returns["ret_portfolio"][1:].std() * np.sqrt(252)

        # sharpe-ratio
        self.sharpe_ratio = {}
        for ix_asset in self.assets:
            ret_col_name = "ret_" + ix_asset
            self.sharpe_ratio[ix_asset] = (self.returns[ret_col_name][1:].mean() / self.returns[ret_col_name][1:].std()) * np.sqrt(252)
        self.sharpe_ratio["portfolio"] = (self.returns["ret_portfolio"][1:].mean() / self.returns["ret_portfolio"][1:].std()) * np.sqrt(252)
        
        # maximum drawdown
        self.drawdown_max = {}
        for ix_asset in self.assets:
            drawdown_col_name = "drawdown_" + ix_asset
            self.drawdown_max[ix_asset] = self.returns[drawdown_col_name].min()
        self.drawdown_max["portfolio"] =  self.returns["drawdown_portfolio"].min()
        
        # annual performance
        df_portfolio = self.returns[["date", "ret_portfolio"]].copy()
        df_portfolio["date"] = pd.to_datetime(df_portfolio["date"])
        df_portfolio["year"] = df_portfolio["date"].dt.year
        self.annual_performance = df_portfolio.groupby(["year"])[["ret_portfolio"]].agg(lambda x: np.prod(1 + x) - 1).reset_index()

    def calc_period_drawdowns(self) -> None:
        """
        Calculates the performance of the weighted portfolio during the drawdown periods.
        """
        drawdowns_portfolio = []
        for ix in self.market_corrections.index:
            dt_start = self.market_corrections.at[ix, "start"]
            dt_end = self.market_corrections.at[ix, "end"]
            drawdown_portfolio = period_max_drawdown(asset="portfolio", date_start=dt_start, date_end=dt_end, df_ret=self.returns)
            drawdowns_portfolio.append(drawdown_portfolio)
        self.market_corrections["drawdown_portfolio"] = drawdowns_portfolio


In [ ]:
portfolio = {
    "spy":0.5,
    "hyg":0.5
}
# portfolio = {
#     "spy": 0.45,
#     "agg": 0.1,
#     "tlt": 0.2,
#     "buffer_010": 0.1,
#     "buffer_020": 0.1,
#     "buffer_100": 0.05,
# }
date_start = datetime.date(2007, 4, 11)
date_end = datetime.date(2024, 12, 31)

In [ ]:
drb = FixedWeightBacktester(portfolio, df_px, df_corrections, date_start, date_end, None)
drb.calc_daily_returns()
drb.calc_portfolio_statistics()
drb.calc_period_drawdowns()

In [ ]:
print(drb.cumulative_return)
print(drb.annual_return)
print(drb.volatility)
print(drb.sharpe_ratio)
print(drb.drawdown_max)

{'spy': 4.698201111828501, 'hyg': 1.2849774775552096, 'portfolio': 2.7857792474375533}
{'spy': 0.10326982171084764, 'hyg': 0.04777623619061333, 'portfolio': 0.07808357049579095}
{'spy': 0.19866942398323126, 'hyg': 0.11237631679457355, 'portfolio': 0.14362456326670317}
{'spy': 0.5942315261922044, 'hyg': 0.471367186926365, 'portfolio': 0.5953930840795643}
{'spy': -0.5518944290604038, 'hyg': -0.3424653847240299, 'portfolio': -0.4419575389552498}


In [ ]:
# drb = FixedWeightBacktester(portfolio, df_px, df_corrections, date_start, date_end, None)
# drb.calc_daily_returns()
# drb.calc_portfolio_statistics()
# drb.calc_period_drawdowns()

In [ ]:
# print(drb.cumulative_return)
# print(drb.annual_return)
# print(drb.volatility)
# print(drb.sharpe_ratio)
# print(drb.drawdown_max)

In [ ]:
# print(drb.portfolio)
# print(drb.assets)
# print(drb.weights)

In [ ]:
#drb.prices.head()

In [ ]:
#drb.market_corrections.head()

In [ ]:
#drb.returns

In [ ]:
#drb.annual_performance

In [ ]:
#drb.market_corrections

## OLD CODE

In [ ]:
# def get_market_data(self):
#     # getting asset prices
#     pf = PriceFetcher(assets=[x.upper() for x in self.assets])
#     pf.fetch()
#     self.prices = pf.prices
#     pf = None

#     # getting market corrections
#     mc = MarketCorrections(
#         asset=self.reference_market.upper(),
#         correction=self.correction_size,
#     )
#     self.market_corrections = \
#         mc.corrections.query("@self.date_start <= start & end <= @self.date_end")